In [1]:
import re

def parse_numbers(text):
    """
    예) "[2, 1, 10, 19, 6]"  -> "2, 1, 10, 19, 6"
        "2 1 3"             -> "2, 1, 3"
        "2,    1 3"         -> "2, 1, 3"
    """
    # 1) 대괄호 제거
    text = text.replace('[', '').replace(']', '')

    # 2) 쉼표와 공백을 모두 공백 하나로 통일
    text = re.sub(r'[,\s]+', ' ', text)

    # 3) 공백기준으로 잘라 숫자 배열 얻기
    tokens = text.strip().split()

    # 4) "숫자, 숫자, 숫자" 형태로 합치기
    return ', '.join(tokens)


def parse_user_results(lines):
    """
    lines: 해당 사용자에게 속한 모든 라인(질문+답변 형태)
    반환값: ["Question 1: ...", "Question 2: ...", ...] 형태의 리스트
    """

    user_results = []

    # "다음 줄에서 숫자 목록을 기대하고 있는" 질문 번호
    pending_question_num = None

    # "숫자 목록만 등장했을 때" 자동으로 붙일 질문 번호를 세는 카운터
    auto_question_counter = 1

    # 괄호([])까지 포함해서 숫자/쉼표/공백만 있는지 확인
    numbers_pattern = re.compile(r'^[\[\]\d,\s]+$')

    # Question 패턴 (1) "Question 2) blah" (2) "2) blah" (3) "2: blah" (4) "2. blah"
    question_pattern = re.compile(r'^(?:Question\s*)?(\d+)(?:\)|:|\.)\s*(.*)$', re.IGNORECASE)

    for raw_line in lines:
        line = raw_line.strip()
        if not line:
            continue

        # ----- 전처리(필요에 따라 변경) -----
        # 전각 쉼표 -> 반각 쉼표
        line = line.replace('，', ',')

        # 별표 제거
        line = re.sub(r'\*+', '', line)

        # (마침표나 쉼표가 여러 개 있는 경우는 parse_numbers에서 처리해도 되지만,
        #  일단 여기서는 기존 로직 유지)
        line = re.sub(r'\.+$', '', line)
        line = re.sub(r',+$', '', line)

        # 중복공백 정리
        line = re.sub(r'\s+', ' ', line).strip()
        # -----------------------------------

        if not line:
            continue

        # (1) "Question N) ..." 패턴인지 확인
        q_match = question_pattern.match(line)
        if q_match:
            q_num = q_match.group(1)
            rest = q_match.group(2).strip()

            # rest가 숫자만으로 이뤄진 라인(괄호 포함 가능)이라면 즉시 등록
            if numbers_pattern.match(rest):
                # 쉼표 형태로 표준화
                answer_str = parse_numbers(rest)
                user_results.append(f"Question {q_num}: {answer_str}")
            else:
                # 그 외면 다음 라인에서 숫자 목록을 받도록 등록
                pending_question_num = q_num

            continue

        # (2) 직전에 "Question N)"이 있었고, 지금 숫자 목록이라면 연결
        if pending_question_num is not None:
            if numbers_pattern.match(line):
                answer_str = parse_numbers(line)
                user_results.append(f"Question {pending_question_num}: {answer_str}")

            # 질문 번호 소진
            pending_question_num = None
            continue

        # (3) "숫자 목록만" 있는 줄이면 자동 Question 할당
        if numbers_pattern.match(line):
            answer_str = parse_numbers(line)
            user_results.append(f"Question {auto_question_counter}: {answer_str}")
            auto_question_counter += 1
            continue

        # (4) 그 밖에 라인은 무시(필요 시 수정)
        continue

    # 결과 리스트 반환!
    return user_results    


def standardize_results(input_filename, output_filename):
    """
    - input_filename : 원본 텍스트 파일 이름
    - output_filename: 결과를 저장할 텍스트 파일 이름
    """

    input_filename = f'../../results/gpt_result/{input_filename}'
    output_filename = f'../../results/gpt_result/{output_filename}'

    with open(input_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    user_data = {}
    current_user = None
    current_lines = []

    for line in lines:
        line_stripped = line.strip()
        if not line_stripped:
            continue

        # 사용자 식별자([U1], [U2], ...) 탐색
        user_match = re.match(r'\[([^\]]+)\]', line_stripped)
        if user_match:
            # 이전 사용자의 데이터 파싱 확정
            if current_user is not None and current_lines:
                user_data[current_user] = parse_user_results(current_lines)
                current_lines = []

            # 새 사용자 식별
            current_user = user_match.group(1)
        else:
            # 사용자 구간 내부라면 라인을 수집
            if current_user is not None:
                current_lines.append(line_stripped)

    # 마지막 사용자 처리
    if current_user and current_lines:
        user_data[current_user] = parse_user_results(current_lines)

    # 결과 쓰기
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        for user_id, results in user_data.items():
            outfile.write(f"[{user_id}]\n")
            for res_line in results:
                outfile.write(f"{res_line}\n")
            outfile.write("\n")


In [24]:
# input_filename = '[250217] negative_ndcg.txt'
# output_filename = '[250217] negative_ndcg3.txt'
# standardize_results(input_filename, output_filename)

input_filename = '[250217] positive_ndcg_few shot.txt'
output_filename = '[250217] positive_ndcg_few shot3.txt'
standardize_results(input_filename, output_filename)

input_filename = '[250217] positive_ndcg_no few shot.txt'
output_filename = '[250217] positive_ndcg_no few shot3.txt'
standardize_results(input_filename, output_filename)

In [8]:
# input_filename = '[250220] negative_acc.txt'
# output_filename = '[250220] negative_acc3.txt'
# standardize_results(input_filename, output_filename)

input_filename = '[250225-ranking] negative_fine.txt'
output_filename = '[250225-ranking] negative_fine2.txt'
standardize_results(input_filename, output_filename)